In [1]:
import pandas as pd

class SharePriceProcessor:
    """
    A class to process, transform, and save share price data.
    """

    def __init__(self):
        """
        Initialize the processor with the hardcoded file path to the raw data.
        """
        self.filepath = 'data/raw/de_share_prices_data_RAW.csv'
        self.raw_prices = None
        self.mode_shares = None

    def load_data(self):
        """
        Load the raw share price data from the file.
        """
        self.raw_prices = pd.read_csv(self.filepath)
        self.raw_prices['Date'] = pd.to_datetime(self.raw_prices['Date'])

    def extract_date_features(self, df):
        """
        Extract features from the Date column.

        Args:
            df (pd.DataFrame): The DataFrame containing the Date column.
        """
        df['Day_of_Week'] = df['Date'].dt.day_name()
        df['Month'] = df['Date'].dt.month
        df['Year'] = df['Date'].dt.year
        df['Day_of_Month'] = df['Date'].dt.day

    def drop_columns(self):
        """
        Drop unnecessary columns from the data.
        """
        if 'Dividend' in self.raw_prices.columns:
            self.raw_prices = self.raw_prices.drop(columns=['Dividend'])

    def calculate_mode_shares(self):
        """
        Calculate the mode of Shares Outstanding grouped by Ticker, Year, and Month.
        """
        self.mode_shares = (
            self.raw_prices.groupby(['Ticker', 'Year', 'Month'])['Shares Outstanding']
            .apply(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
        )

    def fill_missing_values(self):
        """
        Fill missing values in the DataFrame using forward fill and backward fill.
        """
        self.raw_prices.ffill(inplace=True)
        self.raw_prices.bfill(inplace=True)

    def transform_data(self):
        """
        Transforms the data by handling missing values, creating a percentage change
        column, categorizing price movements, and extracting features from the Date column.
        """
        try:
            # Fill missing values
            self.fill_missing_values()

            # Extract features from Date
            self.extract_date_features(self.raw_prices)

            # Initialize rolling window size
            rolling_window = 5

            # Define a categorization function
            def categorize_change(change, percentile_25, percentile_50, percentile_75):
                if change >= percentile_75:
                    return 'High Rise'
                elif percentile_50 <= change < percentile_75:
                    return 'Low Rise'
                elif -0.5 <= change <= 0.5:
                    return 'Stay'
                elif percentile_25 <= change < percentile_50:
                    return 'Low Fall'
                else:
                    return 'High Fall'

            # Process each ticker group separately
            processed_data = []
            for ticker, group in self.raw_prices.groupby('Ticker'):
                group = group.sort_values(by="Date")  # Ensure sorted by date

                # Calculate percentage change
                group['Price_Change'] = group['Close'].pct_change() * 100

                # Calculate rolling percentiles
                group['25th_Percentile'] = group['Price_Change'].rolling(window=rolling_window).quantile(0.25)
                group['50th_Percentile'] = group['Price_Change'].rolling(window=rolling_window).quantile(0.50)
                group['75th_Percentile'] = group['Price_Change'].rolling(window=rolling_window).quantile(0.75)

                # Apply categorization function
                group['Category'] = group.apply(
                    lambda row: categorize_change(
                        row['Price_Change'],
                        row['25th_Percentile'],
                        row['50th_Percentile'],
                        row['75th_Percentile']
                    ),
                    axis=1
                )

                # Append processed group
                processed_data.append(group)

            # Combine all processed ticker groups
            self.raw_prices = pd.concat(processed_data, ignore_index=True)

        except Exception as e:
            print(f"Error during transformation: {e}")

    def save_data(self):
        """
        Save the transformed data into a CSV file with a hardcoded path.
        """
        output_file = 'data/processed/de_share_prices_processed.csv'

        try:
            self.raw_prices.to_csv(output_file, index=False)
            print(f"Transformed data saved to {output_file}")
        except Exception as e:
            print(f"Error during saving: {e}")

    def process_data(self):
        """
        Run all processing steps on the raw data and save the transformed data.
        """
        self.load_data()
        self.extract_date_features(self.raw_prices)
        self.drop_columns()
        self.calculate_mode_shares()
        self.fill_missing_values()
        self.transform_data()
        self.save_data()


# PROCESS AND SAVE THE DATA
processor = SharePriceProcessor()
processor.process_data()


Transformed data saved to data/processed/de_share_prices_processed.csv


In [2]:
# Step 1: Load the processed data
df = pd.read_csv('data/processed/de_share_prices_processed.csv')

# Step 2: Filter for rows where Ticker is 'BMw.DE'
df_bmw = df[df['Ticker'] == 'BMW.DE']

# Step 3: Display the first 20 rows of the filtered data
df_bmw.head(20)

,Ticker,Date,SimFinId,Open,High,Low,Close,Adj. Close,Volume,Shares Outstanding,Day_of_Week,Month,Year,Day_of_Month,Price_Change,25th_Percentile,50th_Percentile,75th_Percentile,Category
7518,BMW.DE,2019-04-08,825112,81.48,82.99,81.37,82.81,58.69,1325909,660000000.0,Monday,4,2019,8,NaN,NaN,NaN,NaN,High Fall
7519,BMW.DE,2019-04-09,825112,82.38,83.43,82.11,82.46,58.45,1281897,660000000.0,Tuesday,4,2019,9,-0.422654,NaN,NaN,NaN,Stay
7520,BMW.DE,2019-04-10,825112,82.25,82.97,82.19,82.53,58.49,1264318,660000000.0,Wednesday,4,2019,10,0.084890,NaN,NaN,NaN,Stay
7521,BMW.DE,2019-04-11,825112,82.57,83.15,81.94,82.78,58.67,1287851,660000000.0,Thursday,4,2019,11,0.302920,NaN,NaN,NaN,Stay
7522,BMW.DE,2019-04-12,825112,82.63,85.51,82.22,85.22,60.40,2612728,660000000.0,Friday,4,2019,12,2.947572,NaN,NaN,NaN,High Fall
7523,BMW.DE,2019-04-15,825112,84.93,85.90,84.68,85.52,60.60,1175155,660000000.0,Monday,4,2019,15,0.352030,0.084890,0.302920,0.352030,High Rise
7524,BMW.DE,2019-04-16,825112,85.76,86.53,85.46,86.22,61.10,1587145,660000000.0,Tuesday,4,2019,16,0.818522,0.302920,0.352030,0.818522,High Rise
7525,BMW.DE,2019-04-17,825112,87.10,88.16,86.20,87.27,61.85,2357190,660000000.0,Wednesday,4,2019,17,1.217815,0.352030,0.818522,1.217815,High Rise
7526,BMW.DE,2019-04-18,825112,86.74,87.97,86.34,87.36,61.91,2868292,660000000.0,Thursday,4,2019,18,0.103128,0.352030,0.818522,1.217815,Stay
7527,BMW.DE,2019-04-23,825112,87.27,87.76,86.88,86.99,61.64,1139985,660000000.0,Tuesday,4,2019,23,-0.423535,0.103128,0.352030,0.818522,Stay
